In [37]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io #Used to load the OCTAVE *.mat files
import scipy.misc #Used to show matrix as an image
import matplotlib.cm as cm #Used to display images in a specific colormap
import random #To pick random images to display
import math
from scipy.special import expit #Vectorized sigmoid function
import array

In [38]:
dataFile = "data/ex4data1.mat"
data = scipy.io.loadmat(dataFile)

X = data['X']
y = data['y']

X = np.insert(X, 0, 1, axis = 1)

In [111]:
# taken from ex3
def reshapeRow(row):
    """
    @param {row} 1 x 401 matrix since an image of a digit is 20x20 + 1 that was added as a bias
    Function that takes in the pixel intensity values and puts it into a 20x20 square 
    """ 
    # the [1:] is used to take everything after the 1st index 
    
    return row[1:].reshape(20,20).T

def displayData(indiciesToDisplay = None):
    """
    Function that selects 100 random examples for the 5000 we have and organizes
    them into a 10x10 matrix
    """
    width = 20
    height = 20
    numRows = 10
    numCols = 10
    
    if not indiciesToDisplay:
        indiciesToDisplay = random.sample(range(X.shape[0]), numRows * numCols)

    
    bigPicture = np.zeros((height * numRows, width * numCols))
    
    iRow = 0
    iCol = 0

    for i in indiciesToDisplay :
        if iCol == numCols:
            iCol = 0
            iRow += 1
 
        curImg = reshapeRow(X[i])
        bigPicture[iRow * height :iRow * height + curImg.shape[0], 
                    iCol * width : iCol * width + curImg.shape[1]] = curImg
        iCol += 1 
    fig = plt.figure( figsize = (6,6) )
    img = scipy.misc.toimage( bigPicture )
    plt.imshow(img,cmap = cm.Greys_r)

#displayData()

In [112]:
thetaFile = "data/ex4weights.mat"
thetas = scipy.io.loadmat(thetaFile)

# Theta1.shape = 25 x 401
Theta1 = thetas['Theta1']

# Theta2.shape = 10 x 26
Theta2 = thetas['Theta2']

inputSize = 400
hiddenSize = 25
outputSize = 10

# n = 401
n = X.shape[1]

In [113]:
def sigmoid(X, theta):
    return expit(np.dot(X,theta))

def costFunction(X, y, thetas, lmbda):
    # m = 5000
    m = X.shape[0]
    
    totalCost = 0
    for i in range(m):        
        a, z = feedForward(X[i], thetas)
        a3 = a[1]
        # create output array with 1 in the appropriate index
        tempY = np.zeros((10, 1))
        tempY[y[i]-1] = 1
    
        cost = -(tempY.T.dot(np.log(a3)) + (1-tempY.T).dot(np.log(1-a3)))
        totalCost += cost

       
    totalCost = totalCost/m
    totalCost = np.sum(totalCost)
    totalRegularized = 0
    thetas = np.array(thetas)
    for i in thetas:
        totalRegularized += np.sum(np.multiply(i,i))
        
    totalRegularized *= float(lmbda)/(2*m)
    
    return totalCost + totalRegularized

# remember that theta is initialized to random values
# returns a1,a2...aL in matrix a
def feedForward(row, thetas):
    nThetas = len(thetas)
    a = np.array(2)
    aVals = [None]*nThetas
    zVals = [None]*nThetas

    for i in range(nThetas):
        curTheta = thetas[i]
        if (i == 0):
            z = curTheta.dot(row)
            a = expit(z)
            a = np.insert(a,0,1)
            a = a.reshape(len(a), 1)
            aVals[i] = a
            zVals[i] = z
            continue
            
        z = curTheta.dot(a)
        a = expit(z)
        
        # if we're on the last iteration we don't want to add the bias term because
        # the last iteration will be the output
        if (i != nThetas-1):
            a = np.insert(a,0,1)
        aVals[i] = a
        zVals[i] = z
        
    a = a.reshape(len(a), 1)
    return aVals, zVals

In [114]:
def testCost():
    thetas = [Theta1,Theta2]
    J = costFunction(X,y,thetas,1)
    print("Expected value is ~ 0.383\nActual value %.5f"%J)
testCost()

Expected value is ~ 0.383
Actual value 0.38449


In [115]:
def gradientSigmoid(z):
    return expit(z)*(1-expit(z))

def initRandomThetas():
    epsilon = 0.12
    theta1 = (hiddenSize, inputSize + 1)
    theta2 = (outputSize, hiddenSize + 1)
    thetas = [np.random.rand(*theta1)*2*epsilon - epsilon, \
                    np.random.rand(*theta2)*2*epsilon - epsilon]
    return thetas

temp = initRandomThetas()



In [181]:
def backProp(X, y, thetas, lmbda):
    m = X.shape[0]
              
    for i in range(m):
        curRow = X[i].reshape(len(X[i]),1)
        
        a, z = feedForward(curRow,thetas)
        
        # 26 x 1
        a2 = a[0]
        # 10 x 1
        a3 = a[1]
        
        # 25 x 1
        z2 = z[0]
        z2 = z2.reshape(len(z2),1)

        # 10 x 1
        z3 = z[1]
        
        tempY = np.zeros((10,1))
        tempY[y[i] - 1] = 1
        
        # 10 x 1
        delta3 = z3 - tempY
        # remove 1st column of 1's from theta2  
        theta2 = np.delete(thetas[1], -1, axis=1)
        delta2 = theta2.T.dot(delta3)*gradientSigmoid(z2)
        
        if (i == 0):
            Delta1 = delta2.dot(curRow.T)
            Delta2 = delta3.dot(a2.T)
        else:
            Delta1 += delta2.dot(curRow.T)
            Delta2 += delta3.dot(a2.T)
    
    D1 = Delta1/m
    D2 = Delta2/m
    
    # all rows, excluding the 1st column which has ones
    D1[:,1:] = D1[:,1:] + float(lmbda/m)*thetas[0][:,1:]
    D2[:,1:] = D2[:,1:] + float(lmbda/m)*thetas[1][:,1:]
    
    return D1,D2

def printShape(a,name):
    print("%s.shape is (%d,%d)" %(name,a.shape[0],a.shape[1]))

thetas = [Theta1,Theta2]
D1,D2 = backProp(X,y,thetas,0)
    

In [186]:
def checkGradient(X, y, thetas, D, lmbda):
    epsilon = 0.0001
    
    theta1 = thetas[0].ravel()
    theta1 = theta1.reshape(len(theta1),1)
    
    theta2 = thetas[1].ravel()
    theta2 = theta2.reshape(len(theta2),1)
    
    
    thetaUnrolled = np.append(theta1,theta2)
    thetaUnrolled = thetaUnrolled.reshape(len(thetaUnrolled),1)

    for i in range(10):
        x = int(np.random.rand()*len(thetaUnrolled))
        epsilonVector = np.zeros((len(thetaUnrolled),1))
        epsilonVector[x] = epsilon
        
        costHigh = costFunction(X, y, thetaUnrolled + epsilonVector, lmbda)
        costHigh = costFunction(X, y, thetaUnrolled - epsilonVector, lmbda)
        grad = costHigh - costLow
        grad /= float(2*epsilon)
        
        print(grad)
        
        #print("Element: %d. Numerical Gradient = %f. BackProp Gradient = %f." %(x,grad,D[x]))

    

In [187]:
checkGradient(X,y,thetas,[D1,D2],0)

ValueError: shapes (1,) and (401,) not aligned: 1 (dim 0) != 401 (dim 0)